In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import log2

In [ ]:
def import_results(filename):
    try:
        df = pd.read_csv(filename, names=["node", "repetition", "instance", "frame_size", "num_bytes", "iterations", "transmission_time", "use_ack"])
        print("imported " + filename)
    except FileNotFoundError:
        return pd.DataFrame()
    df["average_transmission_time"] = df["transmission_time"] / df["iterations"].values
    df["throughput"] = df["num_bytes"] / df["average_transmission_time"]
    df["throughput_gbit/s"] = df["throughput"] * 8 / 1000000000
    return df

In [ ]:
results = import_results("latency_tests.csv")
results

In [ ]:
def count_results_per_node(results):
    nodes = results['node'].unique()
    nodes.sort() 
    for node in nodes:
        print("{}: {} results".format(node, results[results["node"] == node].shape[0]))

count_results_per_node(results)

In [ ]:
bytesizes = results['num_bytes'].unique()
bytesizes.sort()
framesizes = results['frame_size'].unique()
framesizes.sort()
# move 0 for streaming at the end
framesizes = np.roll(framesizes, -1)

def tabulate(df, col):
    data = [[df[col][df['frame_size'] == f][df['num_bytes'] == b].mean() for f in framesizes] for b in bytesizes] 
    return pd.DataFrame(data, columns=[str(f) for f in framesizes], index=[str(b) for b in bytesizes])

tabulate(results, 'average_transmission_time')

In [ ]:
def export_for_plot(df, frame_size=np.inf):
    df = df[df['frame_size'] == frame_size]
    for b in bytesizes:
        print('{},{},QSFP,Aurora'.format(b, df['average_transmission_time'][df['num_bytes'] == b].mean()))

export_for_plot(results)

In [ ]:
def plot_over_filter_by(df, col, over, filter_by=None, override_title=None):
    title = "Average throughput over {}".format(over)
    if (filter_by is not None):
        for key, val in filter_by.items():
            df = df[df[key] == val]
            title += "\n for {} = {}".format(key, val)
    if override_title is not None:
        title = override_title
    fig, axs = plt.subplots(figsize = (13, 9))
    unique_values = df[over].unique()
    unique_values.sort()
    if (over == 'num_bytes'):
        keys = [str(int(log2(v))) for v in unique_values]
    else:
        unique_values = np.roll(unique_values, -1)
        keys = [str(int(v)) for v in unique_values]
    values = [df[col][df[over] == v].mean() for v in unique_values]
    bar_plot = axs.bar(keys, values)
    axs.set_ylabel("Throughput [Gbit/s]")
    axs.set_xlabel(over)
    bar_labels = ["{:10.2f}".format(v) for v in values]
    for rect, label in zip(axs.patches, bar_labels):
        axs.text(rect.get_x() - (rect.get_width() / 4), rect.get_height() + 1.0, label)
    plt.title(title)
    plt.show()

In [ ]:
plot_over_filter_by(results[results['frame_size'] < 2049], "throughput_gbit/s", 'frame_size', filter_by={'num_bytes': 268435456}, override_title="")

In [ ]:
for frame_size in framesizes:
    plot_over_filter_by(results, "throughput_gbit/s", 'num_bytes', filter_by={'frame_size': frame_size})

In [ ]:
for message_size in bytesizes:
    plot_over_filter_by(results, "throughput_gbit/s", 'frame_size', filter_by={'num_bytes': message_size})

In [ ]:
plot_over_filter_by(results, "throughput_gbit/s", "node")

In [ ]:
def violin_over_nodes(df, key):
    nodes = df['node'].unique()
    nodes.sort()
    fig, axs = plt.subplots(nrows=2, ncols=int(len(nodes) / 2), figsize=(16, 9))
    for i, node in enumerate(nodes):
        data = df[df['node'] == node][key]
        subaxs = axs[i % 2, int(i / 2)]
        subaxs.violinplot(data, points=100, showmeans=True, showextrema=True, showmedians=True)
        subaxs.set_title(node)
        subaxs.set_xticks([])
    for i in range(2):
        axs[i, 0].set_ylabel('Throughput [Gbit/s]')

violin_over_nodes(results, 'throughput_gbit/s')